In [1]:
pip install osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 2.5 MB/s eta 0:00:00


In [2]:
import osmnx as ox
import pandas as pd

In [7]:
def expandir_coordenadas(coordenadas, distancia=350):
    coordenadas_expandidas = []

    # Se crea un DataFrame vacío para almacenar las coordenadas expandidas
    df_expandido = pd.DataFrame(columns=['latitude', 'longitude', 'node_type', 'id'])

    for coord in coordenadas:
        # Obtener el id, la latitud y la longitud de la coordenada actual (con esto se hara math con la fuente original para traer los demas campos)
        id_coordenada = coord['id']
        latitud = coord['latitude']
        longitud = coord['longitude']

        G = ox.graph_from_point((latitud, longitud), dist=distancia, network_type='all')

        # Obtener las coordenadas de los nodos cercanos
        coordenadas_nodos = [(G.nodes[nodo]['y'], G.nodes[nodo]['x']) for nodo in G.nodes()]

        coordenadas_expandidas.extend(coordenadas_nodos)

        # Se crea un DataFrame temporal para las coordenadas expandidas de esta iteración
        df_temporal = pd.DataFrame(coordenadas_nodos, columns=['latitude', 'longitude'])

        # Asignar el id correspondiente a todas las filas del DataFrame temporal
        df_temporal['node_type'] = 'secundaria'
        df_temporal['id'] = id_coordenada

        df_expandido = pd.concat([df_expandido, df_temporal])

    return df_expandido


En este apartado tomo el archivo fuente donde estan los nodos pricipales (Hay que considerar que subi el archivo al ambiente local)

In [8]:
# df = pd.read_csv('/content/drive/MyDrive/Diversa/Urbansafe/urbansafe_v1.csv', sep='|')

df = pd.read_csv('urbansafe.csv', sep='|') # Esta linea de codigo el provisional, considerar la de arriba

inputCoord = []

for _, row in df.iterrows():
    misFilas = {
        'id': row['index'],
        'latitude': row['latitude'],
        'longitude': row['longitude']
    }

    inputCoord.append(misFilas)
    # ids.append(misIds)


# Convertir la lista en un nuevo DataFrame
df_input = pd.DataFrame(inputCoord)
valor = 'principal'

df_input.insert(0, 'node_type', valor)

# Verificar el nuevo DataFrame
df_input.count()
# df_input.loc[df_input['id'] == 30]

node_type    41
id           41
latitude     41
longitude    41
dtype: int64

In [9]:
# En esta linea hago el llamado a la funcion
coordenadas_expandidas = expandir_coordenadas(inputCoord)

In [10]:
df_outPut = pd.DataFrame(coordenadas_expandidas)
df_outPut = df_outPut.loc[:, ['node_type','id', 'latitude', 'longitude']]
df_res = pd.concat([df_input, df_outPut], ignore_index=True)
df_res = df_res.drop_duplicates()
df_res

,node_type,id,latitude,longitude
0,principal,1,-32.961405,-60.684841
1,principal,2,-32.958031,-60.686733
2,principal,3,-32.959480,-60.688495
3,principal,4,-32.960392,-60.687334
4,principal,5,-32.961718,-60.686250
...,...,...,...,...
5296,secundaria,42,6.256553,-75.629464
5297,secundaria,42,6.256489,-75.628648
5298,secundaria,42,6.259010,-75.628599
5299,secundaria,42,6.258732,-75.630838


In [11]:
df_res.loc[df_res['id'] == 30] #Esto solo es una prueba

,node_type,id,latitude,longitude
29,principal,30,-0.209389,-78.517758
3213,secundaria,30,-0.212489,-78.517763
3214,secundaria,30,-0.212334,-78.517867
3215,secundaria,30,-0.212002,-78.518331
3216,secundaria,30,-0.212236,-78.517427
...,...,...,...,...
3387,secundaria,30,-0.211674,-78.518487
3388,secundaria,30,-0.210658,-78.520398
3389,secundaria,30,-0.210554,-78.520725
3390,secundaria,30,-0.212500,-78.519443


In [23]:
columnas = ['index','id', 'country', 'country_code', 'city']
df_orig = df[columnas]
df_final = df_orig.merge(df_res, left_on='index', right_on='id', how='inner')
df_final = df_final.drop(columns=['id_y']).rename(columns={'id_x': 'id'})

In [28]:
df_final.loc[df_final['index'] == 30] # Prueba

,index,id,country,country_code,city,node_type,latitude,longitude
3201,30,ECU30,Ecuador,ECU,Quito,principal,-0.209389,-78.517758
3202,30,ECU30,Ecuador,ECU,Quito,secundaria,-0.212489,-78.517763
3203,30,ECU30,Ecuador,ECU,Quito,secundaria,-0.212334,-78.517867
3204,30,ECU30,Ecuador,ECU,Quito,secundaria,-0.212002,-78.518331
3205,30,ECU30,Ecuador,ECU,Quito,secundaria,-0.212236,-78.517427
...,...,...,...,...,...,...,...,...
3376,30,ECU30,Ecuador,ECU,Quito,secundaria,-0.211674,-78.518487
3377,30,ECU30,Ecuador,ECU,Quito,secundaria,-0.210658,-78.520398
3378,30,ECU30,Ecuador,ECU,Quito,secundaria,-0.210554,-78.520725
3379,30,ECU30,Ecuador,ECU,Quito,secundaria,-0.212500,-78.519443


In [29]:
# Guardar el DataFrame resultante en un archivo CSV (Se guardaran localmente)
df_res.to_csv('nodos_expandidos.csv', sep='|', index=False)